# Introduction

In this guide, we will walk you through building a powerful semantic search engine using Couchbase as the backend database and [CrewAI](https://github.com/joaomdmoura/crewAI) for agent-based RAG operations. CrewAI allows us to create specialized agents that can work together to handle different aspects of the RAG workflow, from document retrieval to response generation. This tutorial is designed to be beginner-friendly, with clear, step-by-step instructions that will equip you with the knowledge to create a fully functional semantic search system from scratch.

# Setting the Stage: Installing Necessary Libraries

To build our semantic search engine, we need a robust set of tools. The libraries we install handle everything from connecting to databases to performing complex machine learning tasks.

In [1]:
%pip install datasets langchain-couchbase langchain-openai crewai python-dotenv tqdm

Note: you may need to restart the kernel to use updated packages.


# Importing Libraries and Setting Up Logging

Import all necessary libraries and configure logging.

In [2]:
import json
import logging
import time
import sys
import os
from datetime import timedelta
from uuid import uuid4
from typing import Any, Optional
from dotenv import load_dotenv

from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.exceptions import InternalServerFailureException, QueryIndexAlreadyExistsException
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from datasets import load_dataset
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_core.globals import set_llm_cache
from langchain_couchbase.cache import CouchbaseCache
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain.tools import Tool
from crewai import Agent, Task, Crew, Process
from tqdm import tqdm

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading Configuration Settings

Load configuration settings from environment variables or use default values.

In [3]:
# Load environment variables
load_dotenv()

# Configuration
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set")

CB_HOST = os.getenv('CB_HOST', 'couchbase://localhost')
CB_USERNAME = os.getenv('CB_USERNAME', 'Administrator')
CB_PASSWORD = os.getenv('CB_PASSWORD', 'password')
CB_BUCKET_NAME = os.getenv('CB_BUCKET_NAME', 'vector-search-testing')
INDEX_NAME = os.getenv('INDEX_NAME', 'vector_search_crew')
SCOPE_NAME = os.getenv('SCOPE_NAME', 'shared')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'crew')
CACHE_COLLECTION = os.getenv('CACHE_COLLECTION', 'cache')

print("Configuration loaded successfully")

Configuration loaded successfully


# Connecting to Couchbase

Connect to Couchbase and set up the required collections.

In [4]:
# Connect to Couchbase
auth = PasswordAuthenticator(CB_USERNAME, CB_PASSWORD)
options = ClusterOptions(auth)
cluster = Cluster(CB_HOST, options)
cluster.wait_until_ready(timedelta(seconds=5))
print("Successfully connected to Couchbase")

# Setup collections
bucket = cluster.bucket(CB_BUCKET_NAME)
bucket_manager = bucket.collections()

# Setup main collection
collections = bucket_manager.get_all_scopes()
collection_exists = any(
    scope.name == SCOPE_NAME and COLLECTION_NAME in [col.name for col in scope.collections]
    for scope in collections
)

if not collection_exists:
    bucket_manager.create_collection(SCOPE_NAME, COLLECTION_NAME)
    print(f"Collection '{COLLECTION_NAME}' created")
else:
    print(f"Collection '{COLLECTION_NAME}' already exists")

# Create primary index
cluster.query(
    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{COLLECTION_NAME}`"
).execute()
print(f"Primary index created for '{COLLECTION_NAME}'")

# Clear collection
cluster.query(
    f"DELETE FROM `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{COLLECTION_NAME}`"
).execute()
print(f"Collection '{COLLECTION_NAME}' cleared")

# Setup cache collection
collection_exists = any(
    scope.name == SCOPE_NAME and CACHE_COLLECTION in [col.name for col in scope.collections]
    for scope in collections
)

if not collection_exists:
    bucket_manager.create_collection(SCOPE_NAME, CACHE_COLLECTION)
    print(f"Collection '{CACHE_COLLECTION}' created")
else:
    print(f"Collection '{CACHE_COLLECTION}' already exists")

# Create primary index for cache
cluster.query(
    f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{CACHE_COLLECTION}`"
).execute()
print(f"Primary index created for '{CACHE_COLLECTION}'")

# Clear cache collection
cluster.query(
    f"DELETE FROM `{CB_BUCKET_NAME}`.`{SCOPE_NAME}`.`{CACHE_COLLECTION}`"
).execute()
print(f"Collection '{CACHE_COLLECTION}' cleared")

Successfully connected to Couchbase
Collection 'crew' already exists
Primary index created for 'crew'
Collection 'crew' cleared
Collection 'cache' already exists
Primary index created for 'cache'
Collection 'cache' cleared


# Setting Up Vector Search Index

Load and create the vector search index in Couchbase.

In [5]:
# Load index definition
with open('crew_index.json', 'r') as file:
    index_definition = json.load(file)

# Setup vector search index
scope_index_manager = cluster.bucket(CB_BUCKET_NAME).scope(SCOPE_NAME).search_indexes()

# Check if index exists
existing_indexes = scope_index_manager.get_all_indexes()
index_exists = any(index.name == INDEX_NAME for index in existing_indexes)

if index_exists:
    print(f"Index '{INDEX_NAME}' already exists")
else:
    search_index = SearchIndex.from_json(index_definition)
    scope_index_manager.upsert_index(search_index)
    print(f"Index '{INDEX_NAME}' created")

Index 'vector_search_crew' already exists


# Setting Up OpenAI Components

Initialize OpenAI embeddings and language model.

In [6]:
# Initialize OpenAI components
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    model="text-embedding-ada-002"
)

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o",
    temperature=0
)

print("OpenAI components initialized")

OpenAI components initialized


# Setting Up Vector Store and Cache

Initialize the vector store and cache components.

In [7]:
# Setup vector store
vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=CB_BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
    index_name=INDEX_NAME,
)
print("Vector store initialized")

# Setup cache
cache = CouchbaseCache(
    cluster=cluster,
    bucket_name=CB_BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=CACHE_COLLECTION,
)
set_llm_cache(cache)
print("Cache initialized")

Vector store initialized
Cache initialized


# Loading Sample Data

Load and process the TREC dataset.

In [8]:
# Load TREC dataset
trec = load_dataset('trec', split='train[:1000]')
print(f"Loaded {len(trec)} samples from TREC dataset")

# Add documents in batches
batch_size = 50
for i in tqdm(range(0, len(trec['text']), batch_size), desc="Loading data"):
    batch = trec['text'][i:i + batch_size]
    documents = [Document(page_content=text) for text in batch]
    uuids = [str(uuid4()) for _ in range(len(documents))]
    vector_store.add_documents(documents=documents, ids=uuids)

print("Sample data loaded into vector store")

Loaded 1000 samples from TREC dataset


Loading data: 100%|██████████| 20/20 [01:01<00:00,  3.09s/it]

Sample data loaded into vector store


# Creating Vector Search Tool

Create a tool for performing vector searches.

In [9]:
# Create vector search tool
search_tool = Tool(
    name="vector_search",
    func=lambda query: "\n\n".join([
        f"Document {i+1}:\n{'-'*40}\n{doc.page_content}"
        for i, doc in enumerate(vector_store.similarity_search(
            query if isinstance(query, str) else str(query.get('query', '')),
            k=8,
            fetch_k=20
        ))
    ]),
    description="""Search for relevant documents using vector similarity.
    Input should be a simple text query string.
    Returns a list of relevant document contents with metadata.
    Use this tool to find detailed information about topics."""
)

print("Vector search tool created")

Vector search tool created


# Creating CrewAI Agents

Create specialized agents for research and writing tasks.

In [10]:
# Custom response template
response_template = """
Analysis Results:
----------------
{{ .Response }}

Sources Used:
------------
{% for tool in .Tools %}
- {{ tool.name }}
{% endfor %}

Confidence Level: {{ .Confidence }}
Analysis Time: {{ .ExecutionTime }}
"""

# Create research agent
researcher = Agent(
    role='Research Expert',
    goal='Find and analyze the most relevant documents to answer user queries accurately',
    backstory="""You are an expert researcher with deep knowledge in information retrieval 
    and analysis. Your expertise lies in finding, evaluating, and synthesizing information 
    from various sources. You have a keen eye for detail and can identify key insights 
    from complex documents. You always verify information across multiple sources and 
    provide comprehensive, accurate analyses.""",
    tools=[search_tool],
    llm=llm,
    verbose=True,
    memory=True,
    allow_delegation=False,
    response_template=response_template
)

# Create writer agent
writer = Agent(
    role='Technical Writer',
    goal='Generate clear, accurate, and well-structured responses based on research findings',
    backstory="""You are a skilled technical writer with expertise in making complex 
    information accessible and engaging. You excel at organizing information logically, 
    explaining technical concepts clearly, and creating well-structured documents. You 
    ensure all information is properly cited, accurate, and presented in a user-friendly 
    manner. You have a talent for maintaining the reader's interest while conveying 
    detailed technical information.""",
    llm=llm,
    verbose=True,
    memory=True,
    allow_delegation=False,
    response_template=response_template
)

print("Agents created successfully")

Agents created successfully


# Testing the Search System

Test the system with some example queries.

In [11]:
# Example queries
queries = [
    "What caused the 1929 Great Depression?",
    "Why do heavier objects fall faster?",
    "How does photosynthesis work?"
]

for query in queries:
    print(f"\nQuery: {query}")
    print("-" * 80)
    
    # Create tasks
    research_task = Task(
        description=f"Research and analyze information relevant to: {query}",
        agent=researcher,
        expected_output="A detailed analysis with key findings and supporting evidence"
    )
    
    writing_task = Task(
        description="Create a comprehensive and well-structured response",
        agent=writer,
        expected_output="A clear, comprehensive response that answers the query",
        context=[research_task]
    )
    
    # Create and execute crew
    crew = Crew(
        agents=[researcher, writer],
        tasks=[research_task, writing_task],
        process=Process.sequential,
        verbose=True,
        cache=True,
        planning=True
    )
    
    start_time = time.time()
    result = crew.kickoff()
    elapsed_time = time.time() - start_time
    
    print(f"\nQuery completed in {elapsed_time:.2f} seconds")
    print("=" * 80)
    print("RESPONSE")
    print("=" * 80)
    print(result)
    
    if hasattr(result, 'tasks_output'):
        print("\n" + "=" * 80)
        print("DETAILED TASK OUTPUTS")
        print("=" * 80)
        for task_output in result.tasks_output:
            print(f"\nTask: {task_output.description[:100]}...")
            print("-" * 40)
            print(f"Output: {task_output.raw}")
            print("-" * 40)
    
    print("=" * 80)

17:38:05 - LiteLLM:INFO: utils.py:2741 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
2024-12-05 17:38:05 - INFO - 
LiteLLM completion() model= gpt-4o-mini; provider = openai



Query: What caused the 1929 Great Depression?
--------------------------------------------------------------------------------
 
[2024-12-05 17:38:05][INFO]: Planning the crew execution


2024-12-05 17:38:14 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:38:14 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
2024-12-05 17:38:14 - INFO - Wrapper: Completed Call, calling success_handler
17:38:14 - LiteLLM:INFO: utils.py:2741 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-12-05 17:38:14 - INFO - 
LiteLLM completion() model= gpt-4o; provider = openai


# Agent: Research Expert
## Task: Research and analyze information relevant to: What caused the 1929 Great Depression?1. Start by clearly defining the focus of the research: the causes of the 1929 Great Depression.
2. Utilize the 'vector_search' tool to find relevant documents. Input the query string: 'causes of the 1929 Great Depression'.
3. Review the search results from the vector_search tool, which will provide a list of relevant documents along with their metadata.
4. Assess the documents for credibility, focusing on those that offer key insights and have a strong basis in historical facts.
5. Extract key findings and supporting evidence from the selected documents, ensuring to take notes on important statistics, quotes, and theorists' viewpoints on the causes.
6. Organize the findings into thematic categories (e.g. stock market crash, banking failures, government policy) to facilitate analysis.
7. Summarize the research findings in a detailed manner, outlining each cause of the G

2024-12-05 17:38:16 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:38:16 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
2024-12-05 17:38:16 - INFO - Wrapper: Completed Call, calling success_handler
2024-12-05 17:38:16 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
17:38:17 - LiteLLM:INFO: utils.py:2741 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-12-05 17:38:17 - INFO - 
LiteLLM completion() model= gpt-4o; provider = openai


...

 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.




2024-12-05 17:40:41 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:40:41 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
2024-12-05 17:40:41 - INFO - Wrapper: Completed Call, calling success_handler
17:40:41 - LiteLLM:INFO: utils.py:2741 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-12-05 17:40:41 - INFO - 
LiteLLM completion() model= gpt-4o; provider = openai




# Agent: Research Expert
## Final Answer: 
Unfortunately, the search attempts did not yield relevant documents specifically about the photosynthesis process. However, I can provide a detailed explanation based on my existing knowledge.

Photosynthesis is a complex process used by plants, algae, and some bacteria to convert light energy, usually from the sun, into chemical energy stored in glucose. This process is crucial for life on Earth as it provides the primary energy source for nearly all ecosystems.

The process of photosynthesis can be divided into two main stages: the light-dependent reactions and the Calvin cycle (light-independent reactions).

1. **Light-Dependent Reactions**: These occur in the thylakoid membranes of the chloroplasts. When chlorophyll, the green pigment in plants, absorbs sunlight, it excites electrons, which are then transferred through a series of proteins known as the electron transport chain. This process generates ATP (adenosine triphosphate) and NADP

2024-12-05 17:40:51 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:40:51 - LiteLLM:INFO: utils.py:890 - Wrapper: Completed Call, calling success_handler
2024-12-05 17:40:51 - INFO - Wrapper: Completed Call, calling success_handler




# Agent: Technical Writer
## Final Answer: 
# Understanding Photosynthesis: A Comprehensive Overview

## Introduction to Photosynthesis

Photosynthesis is a fundamental biological process that enables plants, algae, and certain bacteria to convert light energy, primarily from the sun, into chemical energy stored in glucose. This process is vital for life on Earth as it forms the base of the food chain and is responsible for the production of oxygen, which is essential for the survival of aerobic organisms.

## Key Stages of Photosynthesis

Photosynthesis consists of two main stages: the light-dependent reactions and the Calvin cycle, also known as light-independent reactions. Each stage plays a crucial role in the conversion of solar energy into chemical energy.

### Light-Dependent Reactions

The light-dependent reactions occur in the thylakoid membranes of chloroplasts. These reactions begin when chlorophyll, the green pigment in plants, absorbs sunlight. This absorption of light e